<a href="https://colab.research.google.com/github/SevilAnna/GraduationProject/blob/main/EMU403_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>EMÜ 403 - Graduation Project</h1>

---

In [54]:
# importing libraries for the project
import pandas as pd
import numpy as np
from numpy.random import default_rng
rng = default_rng()
import random
!pip install geneticalgorithm
import geneticalgorithm as ga
from geneticalgorithm import geneticalgorithm as ga
print("done")

done


# Importing the Excel File

Either import it from our PC or from the shared Google Drive

🔴 <font color='red'>Kızçeler, Google Drive'dakini çalıştırın bence</font> 🔴

## Import file from PC

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 13.03.2022_Küçük_örnek.XLSX to 13.03.2022_Küçük_örnek.XLSX


In [ ]:
import io
df = pd.read_excel(io.BytesIO(uploaded["13.03.2022_Küçük_örnek.XLSX"]))

## Import file from Google Drive

In [12]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1k0zZ--H6OepIr1qXGIT7iAHL0xqHLcBe'     # 10QcnE8uktxxG4SU8u1O94MOxqrXea2v2
downloaded = drive.CreateFile({'id': file_id})

In [13]:
downloaded.GetContentFile('070322_Düzenlenme_DişliParçaları.xlsx')  # 13.03.2022_Küçük_örnek
!ls -lha 070322_Düzenlenme_DişliParçaları.xlsx
!pip install -q xlrd

-rw-r--r-- 1 root root 156K Mar 21 17:36 070322_Düzenlenme_DişliParçaları.xlsx


In [14]:
from openpyxl import Workbook,load_workbook
df = pd.read_excel('070322_Düzenlenme_DişliParçaları.xlsx', sheet_name='Anka', header=0)
del df['SÜRE (11 parça için)']
del df['1 tanesi için kaç tane gerek']
del df['PROSES']
del df['Process time']
df

,Makine,İşlem,SETUP,SÜRE (1 parça için),PARÇA
0,D13,Pre-Hardening Öncesi Torna,0.0,1.000000,UM121T900201200 GEAR ENGINE Z35
1,D33,Yıkama,0.0,0.090909,UM121T900201200 GEAR ENGINE Z35
2,D31,Ön Sertleştirme (Isıl İşlem),0.0,0.545455,UM121T900201200 GEAR ENGINE Z35
3,D33,Yıkama,0.0,0.090909,UM121T900201200 GEAR ENGINE Z35
4,D32,Temperleme (Isıl İşlem),0.0,0.454545,UM121T900201200 GEAR ENGINE Z35
...,...,...,...,...,...
315,D12,Yıkama,0.0,0.181818,UM121T900204300 GEAR ALTERNATOR Z13 - 2
316,D69,Balans,0.0,1.000000,UM121T900204300 GEAR ALTERNATOR Z13 - 2
317,D65,MPI,0.0,0.454545,UM121T900204300 GEAR ALTERNATOR Z13 - 2
318,D12,Yıkama,0.0,0.090909,UM121T900204300 GEAR ALTERNATOR Z13 - 2


# Next steps

In [15]:
a_list = np.unique(np.array(df["PARÇA"]))
a_list

array(['UM121T900201200 GEAR ENGINE Z35 ',
       'UM121T900201600 GEAR INTERMEDIATE Z57',
       'UM121T900201700 GEAR Z51 ', 'UM121T900202100 GEAR PROPELLER ',
       'UM121T900203800 Z23 ', 'UM121T900203900 GEAR Z19 ',
       'UM121T900203900 GEAR Z19 - 2',
       'UM121T900204300 GEAR ALTERNATOR Z13 ',
       'UM121T900204300 GEAR ALTERNATOR Z13 - 2'], dtype=object)

In [20]:
num = df["PARÇA"].nunique(a_list[0])
u_num = [0]*num
for i in range(9):
  for item in df["PARÇA"]:
    if item == a_list[i]:
      #print(item)
      u_num[i] += 1
u_num

[32, 32, 64, 32, 32, 32, 32, 32, 32]

In [74]:
processes = []
for i in range(num):
  processes.append([])
#print(processes)

j = 0
k = 0
total = 0
for i in range(len(u_num)):
  total += u_num[i]
  while j < total:
    if j == total:
      k += 1
    #print(i,j,k,total,processes)
    #print(processes[i])
    processes[i].append(j+1)
    j += 1
  #print(total)

for i in processes:
  print(len(i),i)

32 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
32 [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
64 [65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128]
32 [129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160]
32 [161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192]
32 [193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217,

In [ ]:
# Mathematical Model

# Constraints


In [50]:
def RandomList(nn, start = 1, end = 100):
    arr = []
    tmp = random.randint(start, end)
    for x in range(nn):
        while tmp in arr:
            tmp = random.randint(start, end)
        arr.append(tmp)  
    return arr

In [73]:
num_genes = total
num_chro = 8
popu_size = (num_chro,num_genes)
new_popu = []
for i in range(num_chro):
  new_popu.append([])
  new_popu[i] = RandomList(len(df["PARÇA"]),1,len(df["PARÇA"]))  #rng.choice(total+1, total+1, replace=False)

for i in new_popu:
  print(i)

[71, 82, 250, 12, 284, 288, 297, 277, 317, 86, 262, 178, 246, 212, 241, 93, 219, 308, 259, 40, 77, 70, 184, 234, 13, 247, 101, 131, 111, 199, 44, 61, 266, 264, 72, 81, 66, 20, 128, 75, 300, 213, 208, 105, 104, 226, 296, 39, 63, 310, 303, 140, 272, 156, 261, 244, 142, 174, 83, 114, 210, 55, 171, 23, 278, 271, 94, 46, 177, 239, 154, 143, 255, 141, 108, 273, 130, 124, 18, 41, 159, 144, 191, 97, 79, 299, 294, 31, 263, 172, 201, 14, 120, 91, 220, 4, 188, 216, 52, 30, 211, 217, 88, 56, 286, 230, 319, 69, 152, 252, 237, 85, 8, 113, 54, 138, 123, 207, 34, 147, 320, 204, 316, 107, 315, 99, 35, 183, 176, 228, 162, 146, 98, 258, 22, 289, 180, 231, 256, 292, 73, 170, 302, 222, 127, 36, 270, 186, 58, 232, 275, 251, 48, 117, 305, 125, 100, 215, 132, 281, 25, 118, 293, 202, 68, 269, 103, 28, 318, 134, 47, 76, 276, 78, 112, 313, 116, 274, 160, 50, 155, 233, 168, 80, 254, 122, 151, 161, 32, 236, 1, 121, 153, 90, 5, 11, 65, 136, 129, 312, 96, 227, 242, 197, 304, 279, 238, 206, 19, 89, 235, 10, 290, 59, 